In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('startup_funding.csv')

In [3]:
print(df.shape)
df.head()

(2372, 10)


,SNo,Date,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentType,AmountInUSD,Remarks
0,0,01/08/2017,TouchKin,Technology,Predictive Care Platform,Bangalore,Kae Capital,Private Equity,"1,300,000",NaN
1,1,02/08/2017,Ethinos,Technology,Digital Marketing Agency,Mumbai,Triton Investment Advisors,Private Equity,NaN,NaN
2,2,02/08/2017,Leverage Edu,Consumer Internet,Online platform for Higher Education Services,New Delhi,"Kashyap Deorah, Anand Sankeshwar, Deepak Jain,...",Seed Funding,NaN,NaN
3,3,02/08/2017,Zepo,Consumer Internet,DIY Ecommerce platform,Mumbai,"Kunal Shah, LetsVenture, Anupam Mittal, Hetal ...",Seed Funding,"500,000",NaN
4,4,02/08/2017,Click2Clinic,Consumer Internet,healthcare service aggregator,Hyderabad,"Narottam Thudi, Shireesh Palle",Seed Funding,"850,000",NaN


In [4]:
df.drop(columns=['SNo'], inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372 entries, 0 to 2371
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              2372 non-null   object
 1   StartupName       2372 non-null   object
 2   IndustryVertical  2201 non-null   object
 3   SubVertical       1436 non-null   object
 4   CityLocation      2193 non-null   object
 5   InvestorsName     2364 non-null   object
 6   InvestmentType    2371 non-null   object
 7   AmountInUSD       1525 non-null   object
 8   Remarks           419 non-null    object
dtypes: object(9)
memory usage: 166.9+ KB


In [6]:
df.columns[df.isnull().any()]

Index(['IndustryVertical', 'SubVertical', 'CityLocation', 'InvestorsName',
       'InvestmentType', 'AmountInUSD', 'Remarks'],
      dtype='object')

In [7]:
df['InvestmentType'].unique()

array(['Private Equity', 'Seed Funding', 'Debt Funding', nan,
       'SeedFunding', 'PrivateEquity', 'Crowd funding', 'Crowd Funding'],
      dtype=object)

In [8]:
df['AmountInUSD'] = df['AmountInUSD'].str.replace(',', '');
df['AmountInUSD'] = pd.to_numeric(df['AmountInUSD'])

In [11]:
df['Date'] = df['Date'].str.replace(r'/+', '/', regex=True)
df['Date'] = df['Date'].str.replace(r'\.+', '/', regex=True)

In [12]:
df['up_date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Date'] = df['up_date'].dt.strftime('%Y-%m-%d')


In [13]:
df['AmountInUSD'].fillna(0, inplace=True)
df['IndustryVertical'].fillna('', inplace=True)
df['SubVertical'].fillna('', inplace=True)
df['CityLocation'].fillna('', inplace=True)
df['InvestorsName'].fillna('', inplace=True)
df['InvestmentType'].fillna('', inplace=True)

In [14]:
df.drop(columns=['up_date', 'formatted_date'], inplace=True)
df

KeyError: "['formatted_date'] not found in axis"

In [15]:
import os
import pandas as pd
from dotenv import load_dotenv
import psycopg2
load_dotenv()

# Replace these with your database credentials
db_params = {
    'dbname': 'verceldb',
    'user': 'default',
    'password': 'esbrqDx83cyj',
    'host': 'ep-cool-dream-68569514-pooler.ap-southeast-1.postgres.vercel-storage.com',
    'port': 5432
}

In [24]:
# Create a list of tuples with values for multiple rows
values_list = [(
    row['StartupName'],
    row['IndustryVertical'],
    row['SubVertical'],
    row['CityLocation'],
    row['InvestorsName'],
    row['InvestmentType'],
    row['AmountInUSD'],
    row['Date'],
    row['Remarks'],
    1  # Assuming user_id is 1 by default
) for index, row in df.head(500).iterrows()]

# Insert multiple rows in a single query
insert_query = '''
    INSERT INTO StartupInfo (
        startup_name,
        startup_industry_vertical,
        startup_sub_vertical,
        startup_city,
        startup_investor_name,
        startup_investment_type,
        startup_funding,
        startup_date,
        startup_remarks,
        user_id
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

[('TouchKin',
  'Technology',
  'Predictive Care Platform',
  'Bangalore',
  'Kae Capital',
  'Private Equity',
  1300000.0,
  '2017-08-01',
  nan,
  1),
 ('Ethinos',
  'Technology',
  'Digital Marketing Agency',
  'Mumbai',
  'Triton Investment Advisors',
  'Private Equity',
  0.0,
  '2017-08-02',
  nan,
  1),
 ('Leverage Edu',
  'Consumer Internet',
  'Online platform for Higher Education Services',
  'New Delhi',
  'Kashyap Deorah, Anand Sankeshwar, Deepak Jain, Sadashiva NT, Arjun Mehta, Satish Kaul, Anindya Ghose',
  'Seed Funding',
  0.0,
  '2017-08-02',
  nan,
  1),
 ('Zepo',
  'Consumer Internet',
  'DIY Ecommerce platform',
  'Mumbai',
  'Kunal Shah, LetsVenture, Anupam Mittal, Hetal Sonpal',
  'Seed Funding',
  500000.0,
  '2017-08-02',
  nan,
  1),
 ('Click2Clinic',
  'Consumer Internet',
  'healthcare service aggregator',
  'Hyderabad',
  'Narottam Thudi, Shireesh Palle',
  'Seed Funding',
  850000.0,
  '2017-08-02',
  nan,
  1),
 ('Billion Loans',
  'Consumer Internet',
  

In [25]:
values_list

# Connect to the PostgreSQL database
connection = psycopg2.connect(**db_params)

# Create a cursor object to interact with the database
cursor = connection.cursor()
print("Database Connected!")

cursor.executemany(insert_query, values_list)

# Commit the changes to the database
connection.commit()
print("Data Saved!")

cursor.close()
connection.close()


Database Connected!
Data Saved!
